# Example WE Notebook for lpath

This notebook details steps towards running the whole lpath analysis with an MD simulation. Check our [Sphinx documentation](https://lpath.readthedocs.io) for more up-to-date information about each function.

By Jeremy Leung
Last updated: May 17th, 2023

## Common cells

In [ ]:
# Imports for all steps
import argparse
import numpy
from lpath import discretize, extract, match, lpath

## Discretize

In [ ]:
# Define function to assign states. In this example,
# input_array is a two column dataset: one for Phi, other for Phi.
def assign_dih(input_array):
    """
    This is an example function for mapping a list of features to state IDs. This should be subclassed.

    Parameters
    ----------
    input_array : numpy.ndarray
        An array generated from load_file.

    Returns
    -------
    state_list : list
        A list containing
    """
    state_list = []
    for val in input_array:
        if val[0] >= -180 and val[0] <= -45 and val[1] >= -55 and val[1] <= 30:  # Phi/Psi for Alpha Helix
            state_list.append(0)
        elif val[0] >= 165 and val[0] <= 180 and val[1] >= -55 and val[1] <= 30:
            state_list.append(0)
        elif val[0] >= -170 and val[0] <= -55 and val[1] >= 40 and val[1] <= 100:  # Phi/Psi for C7eq
            state_list.append(1)
        elif val[0] >= 25 and val[0] <= 90 and val[1] >= -55 and val[1] <= 0:  # Phi/Psi for C7ax
            state_list.append(2)
        else:
            state_list.append(-1)

    return state_list

In [ ]:
# Arguments for `discretize` step.
discretize_args = argparse.Namespace(
    we=False,  # Doing standard simulations
    stride=1,  # Loading at stride=1. Increase to add more frames.
    stats=True,  # Output results statistics
    debug=False,  # Debug mode
    out_dir='succ_traj',  # Name of directory to output the trajectories
    input_name='dihedral.npy',  # Input data for state assignment. Something like 'dihedral.npy'.
    extract_input='states.npy',  # Output file name for the state assignment.
    assign_func='assign_dih',  # Assign function that dictates how to assign states
)

In [ ]:
# Run discretize with the parameters defined in the cell above.
discretize.main(discretize_args)

## Extract

In [ ]:
# Arguments for the `extract` step.
extract_args = argparse.Namespace(
    we=False,  # Doing standard simulations
    stride=1,  # Loading at stride=1. Increase to add more frames.
    stats=True,  # Output results statistics
    debug=False,  # Debug mode
    out_dir='succ_traj',  # Name of directory to output the trajectories
    extract_input='states.npy',  # Name of input assign.h5 file
    extract_output='succ_traj/output.pickle',  # Name of input assign.h5 file

    source_state_num=0,  # Index of the source state as defined in assign_name.
    target_state_num=1,  # Index of the target state as defined in assign_name.
    pcoord=False,  # Option to output extra datasets
    featurization_name='states.npy',  # Specify a file name if pcoord=True 
    feature_stride=1,  # Option to stride `pcoord`
    exclude_short=0,  # Exclude trajectories shorter than provided value during matching. 0 excludes none.
)

In [ ]:
# Run match with the parameters defined in the cell above.
extract.main(extract_args)

## Pattern Match

In [ ]:
# Define function to assign states. In this example,
# input_array is a two column dataset: one for Phi, other for Phi.
def assign_dih(input_array):
    """
    This is an example function for mapping a list of features to state IDs. This should be subclassed.

    Parameters
    ----------
    input_array : numpy.ndarray
        An array generated from load_file.

    Returns
    -------
    state_list : list
        A list containing
    """
    state_list = []
    for val in input_array:
        if val[0] >= -180 and val[0] <= -45 and val[1] >= -55 and val[1] <= 30:  # Phi/Psi for Alpha Helix
            state_list.append(0)
        elif val[0] >= 165 and val[0] <= 180 and val[1] >= -55 and val[1] <= 30:
            state_list.append(0)
        elif val[0] >= -170 and val[0] <= -55 and val[1] >= 40 and val[1] <= 100:  # Phi/Psi for C7eq
            state_list.append(1)
        elif val[0] >= 25 and val[0] <= 90 and val[1] >= -55 and val[1] <= 0:  # Phi/Psi for C7ax
            state_list.append(2)
        else:
            state_list.append(-1)

    return state_list

In [ ]:
# Define function to group assigned states. In this example,
# input_array is a two column dataset: one for Phi, other for Phi.
def reassign_custom(data, pathways, dictionary, assign_file=None):
    """
    Reclassify/assign frames into different states. This is highly
    specific to the system. If w_assign's definition is suffcient,
    you can proceed with what's made in the previous step
    using `reassign_identity`.

    In this example, the dictionary maps state idx to its corresponding `state_string`.
    I suggest using alphabets as states.

    Parameters
    ----------
    data : list
        An array with the data necessary to reassign, as extracted from `output.pickle`.

    pathways : numpy.ndarray
        An empty array with shapes for iter_id/seg_id/state_id/pcoord_or_auxdata/weight.

    dictionary : dict
        An empty dictionary obj for mapping `state_id` with `state string`.

    assign_file : str, default : None
        A string pointing to the assign.h5 file. Needed as a parameter for all functions,
        but is ignored if it's an MD trajectory.

    Returns
    -------
    dictionary : dict
        A dictionary mapping each state_id (float/int) with a `state string` (character).
    """
    # Other example for grouping multiple states into one.
    for idx, val in enumerate(data):
        # The following shows how you can "merge" multiple states into
        # a single one.
        flipped_val = numpy.asarray(val)[::-1]
        first_contact = numpy.where(flipped_val[:, 3] < 5)[0][0]
        for idx2, val2 in enumerate(flipped_val):
            # ortho is assigned to state 0
            if val2[2] in [1, 3, 4, 6, 7, 9]:
                val2[2] = 0
            # para is assigned to state 1
            elif val2[2] in [2, 5, 8]:
                val2[2] = 1
            # Unknown state is assigned 2
            if idx2 < first_contact:
                val2[2] = 2
            pathways[idx, idx2] = val2

    # Generating a dictionary mapping each state
    dictionary = {0: 'A', 1: 'B', 2: '!'}

    return dictionary

In [ ]:
# Arguments for the `match` step.
match_args = argparse.Namespace(
    we=False,  # Doing standard simulations
    stride=1,  # Loading at stride=1. Increase to add more frames.
    out_dir='succ_traj',  # Output for the distance Matrix
    extract_output='succ_traj/output.pickle',  # Input file name of the pickle from `extract.py`
    output_pickle='succ_traj/pathways.pickle',  # Output file name of the new reassigned pathways from `lpath.match`
    reassign_method='reassign_custom',  # Reassign method. Could be a module to be loaded.
    match_metric='longest_common_subsequence',  # Use the longest common subsequence metric.
    match_vanilla=False, # Whether to use the metric with a correction term
    dmatrix_remake=True,  # Enable to remake the distance Matrix
    dmatrix_save='succ_traj/distmat.npy',  # If dmatrix_remake is False, load this file instead.
    dmatrix_parallel=-1,  # Number of jobs to submit for distance matrix calculation. Set to -1 to use everything.
    dendrogram_threshold=0.5,  # Threshold for the Dendrogram
    dendrogram_show=True,  # Show the Dendrogram using plt.show()
    cl_output='succ_traj/cluster_labels.npy',  # Output path for cluster labels
    clusters=None,  # Cluster index to output... otherwise None --> All
)

In [ ]:
# Run discretize with the parameters defined in the cell above.
match.main(match_args)

## Run All

In [ ]:
# For calling all steps directly. Note all parameters are specified manually here.
import argparse
from lpath.lpath import main

all_args = argparse.Namespace(
    # Common Parameters
    out_dir="succ_traj",  # Name of directory to output the trajectories.
    debug=False,  # Debug mode
    we=False,  # Not analyzing a WE simulation.
    stats=True,  # Output results statistics
    stride=1,  # Loading at stride=1. Increase to add more frames.

    # Discretize Parameters
    input_name='dihedral.npy',  # Input data for state assignment. Something like 'dihedral.npy'.
    extract_input='states.npy',  # Output file name for the state assignment.
    assign_func='assign_dih',  # Assign function that dictates how to assign states
 
    # Extract Parameters
    # Note west_name and assign_name are repeated from above and removed
    extract_output='succ_traj/output.pickle',  # Name of input assign.h5 file
    source_state_num=0,  # Index of the source state as defined in assign_name.
    target_state_num=1,  # Index of the target state as defined in assign_name.
    pcoord=False,  # Option to output extra datasets
    featurization_name='states.npy',  # Specify a file name if pcoord=True 
    feature_stride=1,  # Option to stride `pcoord`
    exclude_short=0,  # Exclude trajectories shorter than provided value during matching. 0 excludes none.

    # Match Parameters
    # Note west_name, assign_name and out_dir are repeated from above and removed
    output_pickle='succ_traj/pathways.pickle',  # Output file name of the new reassigned pathways from `lpath.match`
    reassign_method='reassign_custom',  # Reassign method. Could be a module to be loaded.
    match_metric='longest_common_subsequence',  # Use the longest common subsequence metric.
    match_vanilla=False, # Whether to use the metric with a correction term
    dmatrix_remake=True,  # Enable to remake the distance Matrix
    dmatrix_save='succ_traj/distmat.npy',  # If dmatrix_remake is False, load this file instead.
    dmatrix_parallel=-1,  # Number of jobs to submit for distance matrix calculation. Set to -1 to use everything.
    dendrogram_threshold=0.5,  # Threshold for the Dendrogram
    dendrogram_show=True,  # Show the Dendrogram using plt.show()
    cl_output='succ_traj/cluster_labels.npy',  # Output path for cluster labels
    clusters=None,  # Cluster index to output... otherwise None --> All
)

In [ ]:
lpath.main(all_args)